# Libraries

In [1]:
from nfw_project import config_paths as cfg
import geopandas as gpd


# Imports

In [2]:
roadless = gpd.read_file(cfg.USFS_GPKG, layer='roadless_area')
states = gpd.read_file(cfg.TIGERLINE_GPKG, layer="us_states")
roads = gpd.read_parquet(cfg.TIGERLINE_ALL_ROADS)
usfs_admin_boundary = gpd.read_file(cfg.USFS_GPKG, layer="forest_admin_boundary")



In [6]:
usfs_admin_boundary.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Reproject to an equal-area CRS in meters EPSG:5070

In [3]:
# Reproject to an equal-area CRS in meters
roadless_5070 = roadless.to_crs("EPSG:5070")
usfs_admin_boundary_5070 = usfs_admin_boundary.to_crs("EPSG:5070")
roads_5070 = roads.to_crs("EPSG:5070")


# Preprocess

In [4]:
# Explode multipart geometries into single polygons
# index_parts=True preserves original index as the first level
usfs_parts = usfs_admin_boundary_5070.explode(index_parts=True)

# Turn the MultiIndex into columns so we know which forest each part belonged to
usfs_parts = usfs_parts.reset_index(names=["forest_idx", "part_idx"])
# ^ "forest_idx" = original row, "part_idx" = part number within that forest

In [5]:
# Spatial join: keep only admin polys that intersect roadless
joined = gpd.sjoin(
    usfs_parts,          # left
    roadless_5070[["geometry"]],       # right, just geometry
    how="inner",
    predicate="intersects",
)

# Unique admin polygons with any roadless intersection
usfs_with_roadless = usfs_parts.loc[
    joined.index.unique()
].copy()

# 5 Mile Buffer & Road Clip

In [ ]:
""" five_miles_m = 5 * 1609.344

usfs_with_roadless["geometry"] = usfs_with_roadless.geometry.buffer(five_miles_m)
#usfs_with_roadless.to_file(cfg.ROADLESS_ANALYSIS_GPKG, layer= 'buffered_usfs_admin_boundary')
roads_5070 = roads.to_crs("EPSG:5070")
buffer_union = usfs_with_roadless.dissolve() 
idx = roads_5070.sindex.query(buffer_union.geometry.values[0], predicate="intersects")
roads_cand = roads_5070.iloc[idx].copy()

print("Total roads:", len(roads_5070))
print("Candidate roads:", len(roads_cand))
roads_cand.to_file(cfg.ROADLESS_ANALYSIS_GPKG, layer="UnClipped_TL_All_Roads", driver="GPKG")
roads_to_clip = roads_cand.copy()
clipped = gpd.clip(roads_to_clip, buffer_union)
 """

# NEXT CELL 294 minutes (4.9 hours)

In [ ]:
# clipped = gpd.clip(roads_cand, buffer_union)
# clipped.to_file(cfg.ROADLESS_ANALYSIS_GPKG, layer="Clipped_TL_All_Roads_Python", driver="GPKG")

# 1 Mile Buffer & Road Clip

In [7]:
miles_m = 1 * 1609.344

usfs_with_roadless["geometry"] = usfs_with_roadless.geometry.buffer(miles_m)
usfs_with_roadless.to_file(cfg.ROADLESS_ANALYSIS_GPKG, layer= 'buffer1mi_usfs_admin_boundary')


buffer_union = usfs_with_roadless.dissolve() 

roads_to_clip = gpd.read_file(cfg.ROADLESS_ANALYSIS_GPKG, layer="Clipped_TL_All_Roads_Python")
clipped = gpd.clip(roads_to_clip, buffer_union)
clipped.to_file(cfg.ROADLESS_ANALYSIS_GPKG, layer="TL_All_Roads_1mibuffer", driver="GPKG")
print("Wrote layer 'TL_All_Roads_1mibuffer' to:", cfg.ROADLESS_ANALYSIS_GPKG)